In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# matplotlib日本語対応
import matplotlib
matplotlib.rc('font', family='BIZ UDGothic')

import pandas_profiling as pdp

## Read Data

### train.csv
|特徴量|内容|例|
|:---:|:---|:---:|
|kind|野菜の種類|だいこん、かぶ、にんじん|
|date|日付|20051107|
|amount|卸売数量合計（合計の意味は、複数産地のものがあればその合計をとっているの意味）||
|mode_price|卸売価格中央値（中央値の意味は、複数産地のものがあればその中央値をとっているの意味）<-最頻値の間違い？||
|area|産地。複数産地のものが売られていた場合は"_"で結合||

- area:各地は「特定できない産地」という意味。「千葉_青森_各地」となっている場合の「各地」は、「千葉でも青森でもないエリア(特定できない)」の意味。

### wather.csv
|特徴量|内容|例|
|:---:|:---|:---:|
|date|日付||
|mean_temp|平均気温（℃）||
|max_temp|最高気温（℃）||
|max_temp_time|最高気温（℃）（時分）||
|min_temp|最低気温（℃）||
|min_temp_time|最低気温（℃）（時分）||
|sum_rain|降水量合計（mm）||
|sun_time|日照時間（時間）||
|mean_humid|平均湿度（%）||
|area|エリア||

In [2]:
base_path = 'data/'

# 野菜データ
vege_df = pd.read_csv(base_path + 'train.csv')

# 天気データ
weather_df = pd.read_csv(base_path + 'weather.csv')

# 野菜データ test
test_df = pd.read_csv(base_path + 'test.csv')

In [3]:
all_df = pd.concat([vege_df, test_df]) # vege_df.append(test_df)でもよいがconcatのほうが早い

## 前処理・加工：天気

In [4]:
weather_df.head(5)

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森


In [5]:
# 日付、温度観測時点を年月日のdatetimeに変換
weather_df['date'] = pd.to_datetime(weather_df['date'], format="%Y%m%d")
weather_df['max_temp_time'] = pd.to_datetime(weather_df['max_temp_time'].str[:-5])
weather_df['min_temp_time'] = pd.to_datetime(weather_df['min_temp_time'].str[:-5])

In [6]:
print("温度観測時点と日付に乖離があるか確認:",(weather_df['date'] - weather_df['max_temp_time']).astype('str').str[:-5].unique())
# display(weather_df[(weather_df['date'] - weather_df['max_temp_time']).astype('str').str[:-5] == ""])

print("温度観測時点と日付に乖離があるか確認:",(weather_df['date'] - weather_df['min_temp_time']).astype('str').str[:-5].unique())
# display(weather_df[(weather_df['date'] - weather_df['min_temp_time']).astype('str').str[:-5] == ""])

温度観測時点と日付に乖離があるか確認: ['0' '-1' '']
温度観測時点と日付に乖離があるか確認: ['0' '-1' '']


In [7]:
# 時点を削除
weather_df.drop(["max_temp_time", "min_temp_time"], axis=1, inplace=True)

In [8]:
# 全国平均を作成
weather_all = weather_df.groupby('date').agg('mean').reset_index()
weather_all['area'] = "全国"
weather_all

,date,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,area
0,2004-11-06,15.525000,20.903125,11.409375,0.437500,6.915625,71.81250,全国
1,2004-11-07,15.337500,20.575000,10.506250,0.000000,8.134375,68.40625,全国
2,2004-11-08,15.018750,20.215625,10.378125,0.015625,6.325000,70.43750,全国
3,2004-11-09,15.687500,21.812500,10.881250,0.234375,7.746875,71.56250,全国
4,2004-11-10,16.478125,21.715625,11.400000,5.453125,5.709375,71.28125,全国
...,...,...,...,...,...,...,...,...
6380,2022-04-26,20.146875,23.943750,16.743750,19.718750,0.546875,85.00000,全国
6381,2022-04-27,19.634375,24.268750,15.193750,5.406250,3.331250,77.21875,全国
6382,2022-04-28,16.965625,21.950000,12.609375,0.062500,7.793750,61.93750,全国
6383,2022-04-29,13.887500,18.603125,9.593750,28.453125,1.031250,78.15625,全国


In [9]:
# 全国平均を統合
weather_df = weather_df.append(weather_all)

In [10]:
area_pairs = vege_df['area'].unique()

yasai_areas = set()

for area_pair in area_pairs:
    areas = area_pair.split('_')
    yasai_areas |= set(areas)

yasai_areas

{'アメリカ',
 'カナダ',
 'トンガ',
 'ニュージーランド',
 'メキシコ',
 '中国',
 '佐賀',
 '兵庫',
 '北海道',
 '千葉',
 '各地',
 '和歌山',
 '埼玉',
 '宮城',
 '宮崎',
 '山形',
 '山梨',
 '岩手',
 '徳島',
 '愛媛',
 '愛知',
 '新潟',
 '東京',
 '栃木',
 '沖縄',
 '熊本',
 '神奈川',
 '福岡',
 '福島',
 '秋田',
 '群馬',
 '茨城',
 '長崎',
 '長野',
 '青森',
 '静岡',
 '香川',
 '高知',
 '鹿児島'}

In [11]:
weather_areas = weather_df['area'].unique()

In [12]:
# エリア返還のmap作成
area_map = {}

update_area_map = {
    '岩手':'盛岡','宮城':'仙台','静岡':'浜松','沖縄':'那覇','神奈川':'横浜','愛知':'名古屋','茨城':'水戸','北海道':'帯広','各地':'全国',
    '兵庫':'神戸','香川':'高松','埼玉':'熊谷','国内':'全国','山梨':'甲府','栃木':'宇都宮','群馬':'前橋','愛媛':'松山'
}

for yasai_area in yasai_areas:
    if yasai_area not in weather_areas and yasai_area not in update_area_map:
        area_map[yasai_area] = '全国' # 外国の天候は全国にしておく
    else:
        area_map[yasai_area] = yasai_area

area_map = {**area_map, **update_area_map}
area_map

{'東京': '東京',
 '福岡': '福岡',
 '群馬': '前橋',
 '宮崎': '宮崎',
 '神奈川': '横浜',
 '中国': '全国',
 'ニュージーランド': '全国',
 '北海道': '帯広',
 '岩手': '盛岡',
 'トンガ': '全国',
 '山梨': '甲府',
 '兵庫': '神戸',
 '栃木': '宇都宮',
 '香川': '高松',
 '山形': '山形',
 '各地': '全国',
 '和歌山': '和歌山',
 '千葉': '千葉',
 '高知': '高知',
 '長崎': '長崎',
 '静岡': '浜松',
 '長野': '長野',
 '秋田': '秋田',
 '鹿児島': '鹿児島',
 '青森': '青森',
 '愛知': '名古屋',
 '熊本': '熊本',
 'アメリカ': '全国',
 '沖縄': '那覇',
 '埼玉': '熊谷',
 '福島': '福島',
 '愛媛': '松山',
 '茨城': '水戸',
 '佐賀': '佐賀',
 '新潟': '新潟',
 'メキシコ': '全国',
 '宮城': '仙台',
 '徳島': '徳島',
 'カナダ': '全国',
 '国内': '全国'}

「各地」「アメリカ」などを「全国」に変換　←精緻化余地あり！！！！

In [13]:
all_df['area'] = all_df['area'].apply(lambda x: '_'.join([area_map[i] for i in x.split('_')]))
display(all_df)

,kind,date,amount,mode_price,area
0,だいこん,20051107,201445.0,735.0,千葉
1,だいこん,20051108,189660.0,840.0,千葉_全国_青森
2,だいこん,20051110,218166.0,735.0,千葉_全国_青森
3,だいこん,20051111,182624.0,682.5,千葉_青森
4,だいこん,20051112,220691.0,682.5,千葉_青森
...,...,...,...,...,...
315,ミニトマト,20220526,NaN,NaN,全国_名古屋
316,ミニトマト,20220527,NaN,NaN,全国_名古屋
317,ミニトマト,20220528,NaN,NaN,全国_名古屋
318,ミニトマト,20220530,NaN,NaN,全国_名古屋


天候データのエリアを「千葉_全国_青森」のように複数エリアにまたがっている場合の各統計量について、各エリアの平均をとることにする

In [14]:
# 卸売りデータのエリアのユニークリストを格納
area_pairs = all_df['area'].unique()

target_cols = [i for i in weather_df.columns if i != 'area']
target_cols_ex_date = [i for i in target_cols if i != 'date']

area_pair_dfs = []

for area_pair in area_pairs:
    areas = area_pair.split('_')
    if len(areas) > 0:
        area = areas[0]
        base_tmp_df = weather_df[weather_df['area'] == area]
        base_tmp_df = base_tmp_df[target_cols].reset_index(drop=True)
        for area in areas[1:]:
            tmp_df = weather_df[weather_df['area'] == area]
            tmp_df = tmp_df[target_cols].reset_index(drop=True)
            base_tmp_df[target_cols_ex_date] = base_tmp_df[target_cols_ex_date].add(tmp_df[target_cols_ex_date])
        base_tmp_df[target_cols_ex_date] /= len(areas)
        base_tmp_df['area'] = area_pair
        area_pair_dfs.append(base_tmp_df)

area_pair_df = pd.concat(area_pair_dfs)
print(area_pair_df.shape)
display(area_pair_df.head())

weather_df = area_pair_df
print("加工後天候データ：",weather_df.shape)

(1545170, 8)


,date,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,area
0,2004-11-06,17.4,21.2,14.1,0.0,5.1,84.0,千葉
1,2004-11-07,17.0,22.2,13.5,0.0,7.9,77.0,千葉
2,2004-11-08,15.8,19.0,12.7,0.0,0.6,72.0,千葉
3,2004-11-09,16.8,21.5,12.4,0.0,7.8,75.0,千葉
4,2004-11-10,16.7,21.9,11.5,0.0,6.7,79.0,千葉


加工後天候データ： (1545170, 8)


売上データと天候データを統合

In [15]:
all_df.date = pd.to_datetime(all_df.date, format="%Y%m%d")
display(all_df)
all_df = all_df.merge(weather_df, on=['date', 'area'], how='left')
display(all_df)

,kind,date,amount,mode_price,area
0,だいこん,2005-11-07,201445.0,735.0,千葉
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森
...,...,...,...,...,...
315,ミニトマト,2022-05-26,NaN,NaN,全国_名古屋
316,ミニトマト,2022-05-27,NaN,NaN,全国_名古屋
317,ミニトマト,2022-05-28,NaN,NaN,全国_名古屋
318,ミニトマト,2022-05-30,NaN,NaN,全国_名古屋


,kind,date,amount,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid
0,だいこん,2005-11-07,201445.0,735.0,千葉,18.800000,24.400000,14.700000,7.000000,5.500000,64.00000
1,だいこん,2005-11-08,189660.0,840.0,千葉_全国_青森,13.891667,18.944792,9.200000,6.156250,6.641667,58.65625
2,だいこん,2005-11-10,218166.0,735.0,千葉_全国_青森,11.723958,16.320833,7.609375,4.067708,6.051042,64.03125
3,だいこん,2005-11-11,182624.0,682.5,千葉_青森,11.150000,14.150000,8.100000,0.750000,0.200000,76.50000
4,だいこん,2005-11-12,220691.0,682.5,千葉_青森,10.900000,14.950000,8.900000,5.250000,2.900000,61.00000
...,...,...,...,...,...,...,...,...,...,...,...
98097,ミニトマト,2022-05-26,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
98098,ミニトマト,2022-05-27,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
98099,ミニトマト,2022-05-28,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN
98100,ミニトマト,2022-05-30,NaN,NaN,全国_名古屋,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
vege_df, test_df = all_df.iloc[:vege_df.shape[0]], all_df.iloc[vege_df.shape[0]:]

前処理・加工後データ保存

In [17]:
vege_df.to_csv(base_path + 'preprocessed_train.csv', index=False)
test_df.to_csv(base_path + 'preprocessed_test.csv', index=False)
weather_df.to_csv(base_path + 'preprocessed_weather.csv', index=False)